In [1]:
import os
import glob
import mne
import numpy as np
import pandas as pd

In [7]:
import os, glob, time
import mne
import numpy as np
import pandas as pd

# ===== Logging helpers =====
t0 = time.perf_counter()
def log(msg):
    now = time.perf_counter() - t0
    print(f"[{now:7.2f}s] {msg}", flush=True)

mne.set_log_level('WARNING')  # reduce console noise

# ============== Config ==============
# Paths
data_dir = r"C:\\Users\\bhdib\\OneDrive\\Desktop\\Foreperiod-Consolidated\\Foreperiod-Consolidated\\BDF"
save_dir = r"C:\\Users\\bhdib\\OneDrive\\Desktop\\Foreperiod-Consolidated\\Foreperiod-Consolidated\\FIF"
os.makedirs(save_dir, exist_ok=True)

# Triggers (GO cue codes)
short_cue = 2
long_cue  = 5
surprise_cue = 8
cue_triggers = {short_cue, long_cue, surprise_cue}
event_id = {"short": short_cue, "long": long_cue, "surprise": surprise_cue}

# ICA settings
RUN_ICA = True
ICA_METHOD = 'infomax'     # 'fastica' also fine if available
ICA_N_COMPONENTS = 20
ICA_HP = 1.0
ICA_LP = 40.0
ICA_RANDOM_STATE = 97

# Speed-ups for ICA
ICA_RESAMPLE_HZ   = 256
ICA_MAX_DUR_SEC   = 600
ICA_FILTER_LENGTH = '10s'

# Rejection thresholds (µV), used AFTER filtering (epochs)
PTP_REJ_PRIMARY_UV  = 300.0   # stricter
PTP_REJ_FALLBACK_UV = 500.0   # fallback if everything rejects
BAD_CHAN_MEDIAN_PTP_UV = 2000.0  # mark channels as bad if median ptp above this

def code_to_name(code):
    for name, val in event_id.items():
        if code == val:
            return name
    return None

def get_stim_channel_name(raw):
    stim_idx = mne.pick_types(raw.info, stim=True, exclude=[])
    if len(stim_idx) > 0:
        return raw.ch_names[int(stim_idx[0])]
    for cand in ["Status", "STATUS", "STI 014", "STI101", "TRIGGER", "Trigger"]:
        if cand in raw.ch_names:
            return cand
    for ch in raw.ch_names:
        lo = ch.lower()
        if "status" in lo or "sti" in lo or "trig" in lo:
            return ch
    raise RuntimeError("No stim channel found.")

def ptp_reject_mask(epochs, ptp_thresh_uv):
    """Return good-epoch mask using peak-to-peak threshold in µV, after filtering."""
    data = epochs.get_data(picks="eeg")  # volts
    ptp = (data.max(axis=2) - data.min(axis=2)) * 1e6  # µV, shape (n_epochs, n_ch)
    epoch_ptp_max = ptp.max(axis=1)
    bad = epoch_ptp_max > ptp_thresh_uv
    log(f"PTP reject @ {ptp_thresh_uv:.0f} µV: {bad.sum()}/{len(bad)} rejected "
        f"(epoch ptp max: min={epoch_ptp_max.min():.1f}, med={np.median(epoch_ptp_max):.1f}, "
        f"95p={np.percentile(epoch_ptp_max,95):.1f}, max={epoch_ptp_max.max():.1f} µV)")
    return ~bad, ptp

def diagnose_and_drop_bad_channels(epochs, ptp, med_ptp_bad_uv=BAD_CHAN_MEDIAN_PTP_UV):
    """Identify very bad channels via median epoch PTP; drop them."""
    med_ptp_ch = np.median(ptp, axis=0)  # µV per channel
    ch_names = epochs.ch_names
    bads = [ch_names[i] for i, v in enumerate(med_ptp_ch) if v > med_ptp_bad_uv]
    if bads:
        top = sorted([(ch_names[i], med_ptp_ch[i]) for i in range(len(ch_names))],
                     key=lambda x: x[1], reverse=True)[:10]
        log("Top channels by median PTP (µV): " + ", ".join([f"{n}:{v:.0f}" for n,v in top]))
        log(f"Marking bad channels (median PTP > {med_ptp_bad_uv:.0f} µV): {bads}")
        epochs.info['bads'] = sorted(set(epochs.info['bads']) | set(bads))
        epochs = epochs.drop_channels(bads)
    else:
        top = sorted([(ch_names[i], med_ptp_ch[i]) for i in range(len(ch_names))],
                     key=lambda x: x[1], reverse=True)[:10]
        log("Top channels by median PTP (µV): " + ", ".join([f"{n}:{v:.0f}" for n,v in top]))
        log("No channels exceeded bad-channel threshold.")
    return epochs

# ============== Run ==============
bdf_files = glob.glob(os.path.join(data_dir, "*.bdf"))
log(f"Found BDF files: {bdf_files}")

for file_path in bdf_files:
    log(f"--- Processing {file_path} ---")
    subject_id = os.path.basename(file_path).replace(".bdf", "")

    try:
        # Load raw
        log("Loading raw…")
        raw = mne.io.read_raw_bdf(file_path, preload=True, verbose=False)
        sfreq = raw.info['sfreq']
        dur_s = raw.n_times / sfreq
        log(f"Loaded: {len(raw.ch_names)} ch, {sfreq:.1f} Hz, {dur_s/60:.1f} min")

        # Ensure EXG types: EXG1-4 as EOG; EXG5-8 as MISC so they NEVER count as EEG
        for ch in ['EXG1','EXG2','EXG3','EXG4']:
            if ch in raw.ch_names:
                raw.set_channel_types({ch: 'eog'})
        for ch in ['EXG5','EXG6','EXG7','EXG8']:
            if ch in raw.ch_names:
                raw.set_channel_types({ch: 'misc'})

        # Bipolar EOGs
        if all(ch in raw.ch_names for ch in ['EXG2','EXG1']):
            mne.set_bipolar_reference(raw, 'EXG2','EXG1', ch_name='VEOG_L', drop_refs=False, copy=False)
            raw.set_channel_types({'VEOG_L':'eog'})
        if all(ch in raw.ch_names for ch in ['EXG4','EXG3']):
            mne.set_bipolar_reference(raw, 'EXG4','EXG3', ch_name='HEOG', drop_refs=False, copy=False)
            raw.set_channel_types({'HEOG':'eog'})
        log(f"EOGs present: {[ch for ch in ['VEOG_L','HEOG'] if ch in raw.ch_names]}")

        # EEG setup
        raw.set_montage("standard_1020", on_missing='ignore')
        raw.set_eeg_reference('average', projection=True)
        log("Set montage + average ref (proj)")

        # Events
        stim_ch = get_stim_channel_name(raw)
        log(f"Using stim channel: {stim_ch}")
        events = mne.find_events(raw, stim_channel=stim_ch, shortest_event=1, verbose=False)
        cue_events = events[np.isin(events[:,2], list(cue_triggers))]
        log(f"Total cue events: {len(cue_events)}")
        if len(cue_events) < 2:
            log("❌ Not enough cue events, skipping.")
            continue

        # ICA (fit on lightweight copy)
        if RUN_ICA:
            from mne.preprocessing import ICA
            raw_ica = raw.copy().pick('eeg')
            raw_ica.resample(ICA_RESAMPLE_HZ, npad="auto")
            if raw_ica.times[-1] > ICA_MAX_DUR_SEC:
                raw_ica.crop(tmax=ICA_MAX_DUR_SEC)
            raw_ica.filter(ICA_HP, ICA_LP, fir_design='firwin', filter_length=ICA_FILTER_LENGTH, verbose=False)

            raw_corr = raw.copy().pick_types(eeg=True, eog=True)
            raw_corr.resample(ICA_RESAMPLE_HZ, npad="auto")
            if raw_corr.times[-1] > ICA_MAX_DUR_SEC:
                raw_corr.crop(tmax=ICA_MAX_DUR_SEC)
            raw_corr.filter(ICA_HP, ICA_LP, fir_design='firwin', filter_length=ICA_FILTER_LENGTH, verbose=False)

            log("Fitting ICA…")
            ica = ICA(n_components=ICA_N_COMPONENTS, method=ICA_METHOD,
                      random_state=ICA_RANDOM_STATE, max_iter='auto')
            ica.fit(raw_ica, verbose=True)
            log("ICA fitted.")
            exclude = set()
            for ref in [ch for ch in ['VEOG_L','HEOG'] if ch in raw_corr.ch_names]:
                inds, _ = ica.find_bads_eog(raw_corr, ch_name=ref)
                exclude.update(inds)
            ica.exclude = sorted(exclude)
            log(f"ICA exclude comps: {ica.exclude}")

        # Epochs (no baseline yet)
        log("Epoching…")
        epochs = mne.Epochs(raw, events=cue_events, event_id=event_id,
                            tmin=-0.5, tmax=1.5, baseline=None, preload=True, verbose=False)
        log(f"Epochs shape: {len(epochs)} trials, {len(epochs.ch_names)} ch")

        # Apply ICA to epochs (if available), then baseline
        if RUN_ICA and 'ica' in locals():
            log("Applying ICA to epochs…")
            ica.apply(epochs)
            log("ICA applied.")
        log("Applying baseline (-0.2, 0)…")
        epochs.apply_baseline((-0.2, 0))

        # Light band-pass on epochs for stable rejection metrics
        log("Filtering epochs 1–40 Hz for rejection metrics…")
        epochs_filt = epochs.copy().pick('eeg')
        epochs_filt.filter(1., 40., fir_design='firwin', filter_length='5s', verbose=False)

        # Peak-to-peak rejection (primary)
        log("Computing PTP rejection (primary)…")
        good_mask, ptp = ptp_reject_mask(epochs_filt, PTP_REJ_PRIMARY_UV)
        good_epochs = epochs[good_mask]
        log(f"Primary pass: {len(good_epochs)} clean epochs")

        # If nothing clean, diagnose bad channels and try again
        if len(good_epochs) == 0:
            log("Diagnosing bad channels via median PTP…")
            epochs_diag = epochs_filt  # already filtered EEG-only
            epochs_diag = diagnose_and_drop_bad_channels(epochs_diag, ptp, BAD_CHAN_MEDIAN_PTP_UV)

            # Recompute mask on reduced channel set
            data = epochs_diag.get_data()  # volts, EEG-only
            ptp2 = (data.max(axis=2) - data.min(axis=2)) * 1e6
            epoch_ptp_max2 = ptp2.max(axis=1)
            bad2 = epoch_ptp_max2 > PTP_REJ_FALLBACK_UV
            log(f"Retry PTP @ {PTP_REJ_FALLBACK_UV:.0f} µV on reduced channel set: "
                f"{bad2.sum()}/{len(bad2)} rejected (95p={np.percentile(epoch_ptp_max2,95):.1f} µV)")
            good_mask2 = ~bad2

            # Apply the same channel drop to the original (unfiltered) epochs
            kept_chs = epochs_diag.ch_names
            epochs_reduced = epochs.copy().pick_channels(kept_chs, ordered=True)
            good_epochs = epochs_reduced[good_mask2]
            log(f"After bad-channel drop: {len(good_epochs)} clean epochs")

        # As a last resort, adaptive threshold so you at least save something
        if len(good_epochs) == 0:
            epoch_ptp_max = (epochs_filt.get_data().ptp(axis=2) * 1e6).max(axis=1)
            thr_adapt = np.percentile(epoch_ptp_max, 90)  # allow top 10% to be rejected
            log(f"Adaptive thresholding at ~90th percentile: {thr_adapt:.1f} µV")
            good_mask_adapt = epoch_ptp_max <= thr_adapt
            good_epochs = epochs[good_mask_adapt]
            log(f"Adaptive pass: {len(good_epochs)} clean epochs")

        if len(good_epochs) == 0:
            log("❌ Still no clean epochs; skipping save for this subject.")
            continue

        # Sequence labeling
        ev_codes = good_epochs.events[:, 2]
        if len(ev_codes) < 2:
            log("Not enough events for sequence pairs; saving unpaired clean epochs.")
            good_epochs_seq = good_epochs.copy()
            good_epochs_seq.metadata = pd.DataFrame({'sequence': []})
            seq_suffix = "_cue_clean-epo.fif"
        else:
            prev_codes = ev_codes[:-1]
            curr_codes = ev_codes[1:]
            sequence_labels = [f"{code_to_name(p)}_{code_to_name(c)}"
                               for p, c in zip(prev_codes, curr_codes)
                               if code_to_name(p) and code_to_name(c)]
            good_epochs_seq = good_epochs[1:].copy()
            good_epochs_seq.metadata = pd.DataFrame({'sequence': sequence_labels})
            seq_suffix = "_cue_sequential-epo.fif"

        # Save epochs
        save_path = os.path.join(save_dir, f"{subject_id}{seq_suffix}")
        log(f"Saving → {save_path}")
        good_epochs_seq.save(save_path, overwrite=True)
        log(f"✅ Saved {save_path}")

        # Save ICA
        if RUN_ICA and 'ica' in locals():
            ica_path = os.path.join(save_dir, f"{subject_id}-ica.fif")
            ica.save(ica_path, overwrite=True)
            log(f"💾 Saved ICA → {ica_path}")

    except Exception as e:
        log(f"❌ Error processing {file_path}: {e}")


[   0.01s] Found BDF files: ['C:\\\\Users\\\\bhdib\\\\OneDrive\\\\Desktop\\\\Foreperiod-Consolidated\\\\Foreperiod-Consolidated\\\\BDF\\Foreperiod_Jmlah.bdf', 'C:\\\\Users\\\\bhdib\\\\OneDrive\\\\Desktop\\\\Foreperiod-Consolidated\\\\Foreperiod-Consolidated\\\\BDF\\Foreperiod_Sub2.bdf', 'C:\\\\Users\\\\bhdib\\\\OneDrive\\\\Desktop\\\\Foreperiod-Consolidated\\\\Foreperiod-Consolidated\\\\BDF\\FP_03.bdf', 'C:\\\\Users\\\\bhdib\\\\OneDrive\\\\Desktop\\\\Foreperiod-Consolidated\\\\Foreperiod-Consolidated\\\\BDF\\sub-01-foreperiod-12-3.bdf', 'C:\\\\Users\\\\bhdib\\\\OneDrive\\\\Desktop\\\\Foreperiod-Consolidated\\\\Foreperiod-Consolidated\\\\BDF\\Sub_05.bdf', 'C:\\\\Users\\\\bhdib\\\\OneDrive\\\\Desktop\\\\Foreperiod-Consolidated\\\\Foreperiod-Consolidated\\\\BDF\\Sub_06.bdf']
[   0.01s] --- Processing C:\\Users\\bhdib\\OneDrive\\Desktop\\Foreperiod-Consolidated\\Foreperiod-Consolidated\\BDF\Foreperiod_Jmlah.bdf ---
[   0.01s] Loading raw…
[  13.29s] Loaded: 75 ch, 1024.0 Hz, 47.4 min


C:\Users\bhdib\AppData\Local\Temp\ipykernel_25556\3695055556.py:116: RuntimeWarning: The unit for channel(s) EXG5 has changed from V to NA.
  raw.set_channel_types({ch: 'misc'})
C:\Users\bhdib\AppData\Local\Temp\ipykernel_25556\3695055556.py:116: RuntimeWarning: The unit for channel(s) EXG6 has changed from V to NA.
  raw.set_channel_types({ch: 'misc'})
C:\Users\bhdib\AppData\Local\Temp\ipykernel_25556\3695055556.py:116: RuntimeWarning: The unit for channel(s) EXG7 has changed from V to NA.
  raw.set_channel_types({ch: 'misc'})
C:\Users\bhdib\AppData\Local\Temp\ipykernel_25556\3695055556.py:116: RuntimeWarning: The unit for channel(s) EXG8 has changed from V to NA.
  raw.set_channel_types({ch: 'misc'})


[  16.04s] EOGs present: ['VEOG_L', 'HEOG']
[  16.08s] Set montage + average ref (proj)
[  16.09s] Using stim channel: Status
[  16.17s] Total cue events: 756
[  63.82s] Fitting ICA…
Fitting ICA to data using 66 channels (please be patient, this may take a while)
Selecting by number: 20 components
Computing Infomax ICA
Fitting ICA took 85.8s.
[ 149.67s] ICA fitted.
[ 150.77s] ICA exclude comps: [2, 3, 10]
[ 150.77s] Epoching…
[ 153.32s] Epochs shape: 756 trials, 77 ch
[ 153.33s] Applying ICA to epochs…
[ 157.38s] ICA applied.
[ 157.38s] Applying baseline (-0.2, 0)…
[ 157.75s] Filtering epochs 1–40 Hz for rejection metrics…


C:\Users\bhdib\AppData\Local\Temp\ipykernel_25556\3695055556.py:186: RuntimeWarning: filter_length (5121) is longer than the signal (2049), distortion is likely. Reduce filter length or filter a longer signal.
  epochs_filt.filter(1., 40., fir_design='firwin', filter_length='5s', verbose=False)


[ 209.47s] Computing PTP rejection (primary)…
[ 210.16s] PTP reject @ 300 µV: 756/756 rejected (epoch ptp max: min=3071.8, med=3586.5, 95p=5245.4, max=27223.1 µV)
[ 210.99s] Primary pass: 0 clean epochs
[ 210.99s] Diagnosing bad channels via median PTP…
[ 211.00s] Top channels by median PTP (µV): Erg1:3586, Erg2:1036, AF8:204, AF7:195, Fp1:187, Fp2:179, F6:178, Fpz:173, F7:152, FT7:147
[ 211.00s] Marking bad channels (median PTP > 2000 µV): ['Erg1']
[ 212.11s] Retry PTP @ 500 µV on reduced channel set: 756/756 rejected (95p=1310.2 µV)
[ 214.43s] After bad-channel drop: 0 clean epochs
[ 215.20s] Adaptive thresholding at ~90th percentile: 1231.6 µV
[ 218.85s] Adaptive pass: 680 clean epochs
[ 222.42s] Saving → C:\\Users\\bhdib\\OneDrive\\Desktop\\Foreperiod-Consolidated\\Foreperiod-Consolidated\\FIF\Foreperiod_Jmlah_cue_sequential-epo.fif
[ 226.79s] ✅ Saved C:\\Users\\bhdib\\OneDrive\\Desktop\\Foreperiod-Consolidated\\Foreperiod-Consolidated\\FIF\Foreperiod_Jmlah_cue_sequential-epo.fif
[

...

[1201.13s] EOGs present: ['VEOG_L', 'HEOG']
[1201.20s] Set montage + average ref (proj)
[1201.21s] Using stim channel: Status
[1201.29s] Total cue events: 768
[1248.12s] Fitting ICA…
Fitting ICA to data using 66 channels (please be patient, this may take a while)
Selecting by number: 20 components
Computing Infomax ICA


C:\Users\bhdib\AppData\Local\Temp\ipykernel_25556\3695055556.py:160: RuntimeWarning: Using n_components=20 (resulting in n_components_=20) may lead to an unstable mixing matrix estimation because the ratio between the largest (66) and smallest (2.4e-05) variances is too large (> 1e6); consider setting n_components=0.999999 or an integer <= 14
  ica.fit(raw_ica, verbose=True)


Fitting ICA took 177.4s.
[1425.50s] ICA fitted.
[1428.50s] ICA exclude comps: [3, 8, 15]
[1428.51s] Epoching…
[1435.96s] Epochs shape: 768 trials, 77 ch
[1435.97s] Applying ICA to epochs…
[1444.42s] ICA applied.
[1444.42s] Applying baseline (-0.2, 0)…
[1445.44s] Filtering epochs 1–40 Hz for rejection metrics…


C:\Users\bhdib\AppData\Local\Temp\ipykernel_25556\3695055556.py:186: RuntimeWarning: filter_length (5121) is longer than the signal (2049), distortion is likely. Reduce filter length or filter a longer signal.
  epochs_filt.filter(1., 40., fir_design='firwin', filter_length='5s', verbose=False)


[1513.82s] Computing PTP rejection (primary)…
[1514.40s] PTP reject @ 300 µV: 768/768 rejected (epoch ptp max: min=3464.2, med=4377.3, 95p=46411.5, max=209363.8 µV)
[1515.25s] Primary pass: 0 clean epochs
[1515.25s] Diagnosing bad channels via median PTP…
[1515.32s] Top channels by median PTP (µV): Erg1:4377, Erg2:1241, Fp2:107, AF8:105, Fpz:104, Fp1:103, AF7:100, AFz:99, AF4:98, P6:98
[1515.32s] Marking bad channels (median PTP > 2000 µV): ['Erg1']
[1516.30s] Retry PTP @ 500 µV on reduced channel set: 768/768 rejected (95p=1912.2 µV)
[1517.92s] After bad-channel drop: 0 clean epochs
[1518.54s] Adaptive thresholding at ~90th percentile: 1571.3 µV
[1519.74s] Adaptive pass: 691 clean epochs
[1521.37s] Saving → C:\\Users\\bhdib\\OneDrive\\Desktop\\Foreperiod-Consolidated\\Foreperiod-Consolidated\\FIF\Sub_06_cue_sequential-epo.fif
[1523.76s] ✅ Saved C:\\Users\\bhdib\\OneDrive\\Desktop\\Foreperiod-Consolidated\\Foreperiod-Consolidated\\FIF\Sub_06_cue_sequential-epo.fif
[1523.77s] 💾 Saved IC

In [8]:
# === EEG FIF Analysis Pipeline ===
# Loads *_epo.fif files, verifies content, computes ERPs, sequence effects,
# extracts CNV/P3 metrics, and saves per-subject + group summaries and plots.

import os, glob, json, warnings
from pathlib import Path
import numpy as np
import pandas as pd
import mne

warnings.filterwarnings("ignore", category=RuntimeWarning)
mne.set_log_level("WARNING")

# -------------------
# Config
# -------------------
FIF_DIR = r"C:\Users\bhdib\OneDrive\Desktop\Foreperiod-Consolidated\Foreperiod-Consolidated\FIF"
OUT_DIR = os.path.join(FIF_DIR, "ANALYSIS")
os.makedirs(OUT_DIR, exist_ok=True)

# Conditions as saved in event_id
CONDITIONS = ["short", "long", "surprise"]

# ERP windows (seconds, relative to GO)
BASELINE = (-0.2, 0.0)     # already applied, used for metrics anyway
CNV_WIN = (-0.2, 0.0)      # pre-GO (mean amplitude)
P3_WIN  = (0.300, 0.600)   # post-GO (mean amplitude)

# Electrode of interest (if present)
CZ_NAME = "Cz"  # will gracefully fall back to GFP if Cz not present

# Plotting flags
SAVE_SUBJECT_PLOTS = True
SAVE_GROUP_PLOTS   = True

# File matchers
EPOCHS_GLOB = "*_cue_*epo.fif"  # matches both _cue_sequential-epo.fif and _cue_clean-epo.fif
ICA_SUFFIX  = "-ica.fif"

# -------------------
# Helpers
# -------------------
def safe_pick_channel(epochs, ch_name):
    if ch_name in epochs.ch_names:
        return ch_name
    # Try common variants (BioSemi often uppercase)
    for alt in [ch_name, ch_name.upper(), ch_name.lower()]:
        if alt in epochs.ch_names:
            return alt
    return None

def mean_in_window(evoked, tmin, tmax, picks=None):
    """Mean amplitude (µV) in a time window."""
    data = evoked.copy().pick(picks=picks).data  # volts
    times = evoked.times
    mask = (times >= tmin) & (times <= tmax)
    if not mask.any():
        return np.nan
    return (data[:, mask].mean(axis=1) * 1e6).mean()  # µV, mean across channels

def gfp(evoked):
    """Global Field Power (RMS across channels)."""
    return evoked.copy().data.std(axis=0)

def plot_and_save_evoked(evoked_dict, title, outpath):
    try:
        mne.viz.plot_compare_evokeds(evoked_dict, combine='mean', title=title, show=False)
        import matplotlib.pyplot as plt
        plt.tight_layout()
        plt.savefig(outpath, dpi=150)
        plt.close()
    except Exception as e:
        print(f"[plot] Could not save {outpath}: {e}")

def subject_id_from_path(path):
    name = Path(path).name
    # strip any known suffixes to leave subject id
    for suf in ["_cue_sequential-epo.fif", "_cue_clean-epo.fif"]:
        if name.endswith(suf):
            return name.replace(suf, "")
    # fallback: remove extension
    return name.replace(".fif", "")

# -------------------
# Discover files
# -------------------
epoch_paths = sorted(glob.glob(os.path.join(FIF_DIR, EPOCHS_GLOB)))
if not epoch_paths:
    raise FileNotFoundError(f"No *_epo.fif found in {FIF_DIR}")

print(f"Found {len(epoch_paths)} epoch files")

# -------------------
# Per-subject processing
# -------------------
subject_rows = []     # for CSV summary
per_subject_metrics = []  # detailed metrics
all_evokeds_by_cond = {c: [] for c in CONDITIONS}  # for group grand averages
all_evokeds_by_seq  = {}  # dict[sequence_label] -> list of evokeds

for ep_path in epoch_paths:
    subj = subject_id_from_path(ep_path)
    print(f"\n=== Subject: {subj} ===")
    epochs = mne.read_epochs(ep_path, preload=True, verbose=False)

    # Try reading ICA info if present (provenance)
    ica_path = os.path.join(FIF_DIR, f"{subj}{ICA_SUFFIX}")
    ica_info = None
    if os.path.exists(ica_path):
        try:
            ica = mne.preprocessing.read_ica(ica_path, verbose=False)
            ica_info = {"n_components": ica.n_components_, "exclude": list(ica.exclude)}
        except Exception:
            ica_info = None

    # Quick counts per condition
    cond_counts = {c: len(epochs[c]) if c in epochs.event_id else 0 for c in CONDITIONS}
    print("Trial counts:", cond_counts)

    # Check sequence metadata if present
    has_meta = epochs.metadata is not None and "sequence" in epochs.metadata.columns
    seq_counts = {}
    if has_meta:
        seq_counts = epochs.metadata['sequence'].value_counts().to_dict()
        print("Sequence counts:", seq_counts)

    # Build evokeds per condition
    evokeds_cond = {}
    for c in CONDITIONS:
        if c in epochs.event_id and len(epochs[c]) > 0:
            evokeds_cond[c] = epochs[c].average()
            all_evokeds_by_cond[c].append(evokeds_cond[c])

    # Build evokeds per sequence (if metadata)
    evokeds_seq = {}
    if has_meta:
        for seq_label, n in seq_counts.items():
            if n > 0:
                ep_seq = epochs['sequence == "{}"'.format(seq_label)]
                if len(ep_seq) > 0:
                    evokeds_seq[seq_label] = ep_seq.average()
                    all_evokeds_by_seq.setdefault(seq_label, []).append(evokeds_seq[seq_label])

    # Extract metrics at Cz (if present) and GFP for each condition
    cz_pick = safe_pick_channel(epochs, CZ_NAME)
    for label, evk in evokeds_cond.items():
        row = {
            "subject": subj,
            "type": "condition",
            "label": label,
            "n_trials": cond_counts.get(label, 0)
        }
        row["cnv_mean_uV@Cz"] = mean_in_window(evk, *CNV_WIN, picks=[cz_pick]) if cz_pick else np.nan
        row["p3_mean_uV@Cz"]  = mean_in_window(evk, *P3_WIN,  picks=[cz_pick]) if cz_pick else np.nan
        # GFP metrics (mean of GFP over window)
        evk_gfp = gfp(evk)
        times   = evk.times
        row["cnv_gfp_uV"] = (evk_gfp[(times>=CNV_WIN[0]) & (times<=CNV_WIN[1])].mean() * 1e6)
        row["p3_gfp_uV"]  = (evk_gfp[(times>=P3_WIN[0])  & (times<=P3_WIN[1]) ].mean() * 1e6)
        per_subject_metrics.append(row)

    # Sequence metrics (optional, only if metadata)
    for seq_label, evk in evokeds_seq.items():
        row = {
            "subject": subj,
            "type": "sequence",
            "label": seq_label,
            "n_trials": seq_counts.get(seq_label, 0)
        }
        row["cnv_mean_uV@Cz"] = mean_in_window(evk, *CNV_WIN, picks=[cz_pick]) if cz_pick else np.nan
        row["p3_mean_uV@Cz"]  = mean_in_window(evk, *P3_WIN,  picks=[cz_pick]) if cz_pick else np.nan
        evk_gfp = gfp(evk); times = evk.times
        row["cnv_gfp_uV"] = (evk_gfp[(times>=CNV_WIN[0]) & (times<=CNV_WIN[1])].mean() * 1e6)
        row["p3_gfp_uV"]  = (evk_gfp[(times>=P3_WIN[0])  & (times<=P3_WIN[1]) ].mean() * 1e6)
        per_subject_metrics.append(row)

    # Save per-subject plots
    if SAVE_SUBJECT_PLOTS and len(evokeds_cond) > 0:
        plot_and_save_evoked(
            evokeds_cond,
            title=f"{subj} — Conditions",
            outpath=os.path.join(OUT_DIR, f"{subj}_conditions_evoked.png"),
        )
    if SAVE_SUBJECT_PLOTS and len(evokeds_seq) > 0:
        plot_and_save_evoked(
            evokeds_seq,
            title=f"{subj} — Sequences",
            outpath=os.path.join(OUT_DIR, f"{subj}_sequences_evoked.png"),
        )

    # One row per subject with quick counts + ICA info
    subject_rows.append({
        "subject": subj,
        **{f"n_{c}": cond_counts[c] for c in CONDITIONS},
        "has_metadata_sequence": bool(has_meta),
        "unique_sequences": ",".join(sorted(seq_counts.keys())) if seq_counts else "",
        "ica_excluded": ",".join(map(str, ica_info["exclude"])) if ica_info else "",
        "ica_n_components": ica_info["n_components"] if ica_info else "",
        "epochs_file": ep_path,
        "ica_file": ica_path if os.path.exists(ica_path) else ""
    })

# -------------------
# Save per-subject summaries
# -------------------
subj_df = pd.DataFrame(subject_rows).sort_values("subject")
subj_df.to_csv(os.path.join(OUT_DIR, "subjects_summary.csv"), index=False)
print(f"\nSaved: subjects_summary.csv → {OUT_DIR}")

metrics_df = pd.DataFrame(per_subject_metrics)
metrics_df.to_csv(os.path.join(OUT_DIR, "erp_metrics_per_subject.csv"), index=False)
print(f"Saved: erp_metrics_per_subject.csv → {OUT_DIR}")

# -------------------
# Grand averages (group)
# -------------------
# Per-condition grand averages
grand_cond = {}
for c in CONDITIONS:
    if len(all_evokeds_by_cond[c]) == 0:
        continue
    # equalize trial counts for fairness (optional)
    try:
        mne.equalize_epoch_counts([evk for evk in all_evokeds_by_cond[c]], method='mintime')
    except Exception:
        pass
    grand_cond[c] = mne.grand_average(all_evokeds_by_cond[c])

# Per-sequence grand averages
grand_seq = {}
for seq_label, ev_list in all_evokeds_by_seq.items():
    if len(ev_list) == 0:
        continue
    try:
        mne.equalize_epoch_counts(ev_list, method='mintime')
    except Exception:
        pass
    grand_seq[seq_label] = mne.grand_average(ev_list)

# Save group plots
if SAVE_GROUP_PLOTS and len(grand_cond) > 0:
    plot_and_save_evoked(
        grand_cond,
        title="Grand Averages — Conditions",
        outpath=os.path.join(OUT_DIR, "grand_conditions_evoked.png"),
    )
if SAVE_GROUP_PLOTS and len(grand_seq) > 0:
    # Limit to top N sequences for readability if you have many
    topN = 8
    seq_items = list(grand_seq.items())[:topN]
    plot_and_save_evoked(
        dict(seq_items),
        title=f"Grand Averages — Sequences (first {topN})",
        outpath=os.path.join(OUT_DIR, "grand_sequences_evoked.png"),
    )

# -------------------
# Group metrics table (CNV/P3 from grand averages)
# -------------------
group_rows = []
def metric_row_from_evoked(label, kind, evk):
    cz = CZ_NAME if CZ_NAME in evk.ch_names else safe_pick_channel(evk, CZ_NAME)
    row = {
        "type": kind, "label": label,
        "cnv_mean_uV@Cz": mean_in_window(evk, *CNV_WIN, picks=[cz]) if cz else np.nan,
        "p3_mean_uV@Cz":  mean_in_window(evk, *P3_WIN,  picks=[cz]) if cz else np.nan,
    }
    evk_gfp = gfp(evk); times = evk.times
    row["cnv_gfp_uV"] = (evk_gfp[(times>=CNV_WIN[0]) & (times<=CNV_WIN[1])].mean() * 1e6)
    row["p3_gfp_uV"]  = (evk_gfp[(times>=P3_WIN[0])  & (times<=P3_WIN[1]) ].mean() * 1e6)
    return row

for c, evk in grand_cond.items():
    group_rows.append(metric_row_from_evoked(c, "condition", evk))
for s, evk in grand_seq.items():
    group_rows.append(metric_row_from_evoked(s, "sequence", evk))

if group_rows:
    pd.DataFrame(group_rows).to_csv(os.path.join(OUT_DIR, "erp_metrics_group.csv"), index=False)
    print(f"Saved: erp_metrics_group.csv → {OUT_DIR}")

print("\nAll done. Check the ANALYSIS folder for CSVs and plots.")


Found 6 epoch files

=== Subject: FP_03 ===
Trial counts: {'short': 317, 'long': 323, 'surprise': 71}
Sequence counts: {'long_long': 156, 'short_short': 149, 'long_short': 139, 'short_long': 134, 'short_surprise': 35, 'surprise_long': 33, 'surprise_short': 29, 'long_surprise': 28, 'surprise_surprise': 8}


C:\Users\bhdib\AppData\Local\Temp\ipykernel_25556\2751598477.py:69: UserWarning: The figure layout has changed to tight
  plt.tight_layout()



=== Subject: Foreperiod_Jmlah ===
Trial counts: {'short': 305, 'long': 309, 'surprise': 65}
Sequence counts: {'long_long': 145, 'short_long': 140, 'long_short': 140, 'short_short': 130, 'short_surprise': 35, 'surprise_short': 35, 'surprise_long': 24, 'long_surprise': 24, 'surprise_surprise': 6}


C:\Users\bhdib\AppData\Local\Temp\ipykernel_25556\2751598477.py:69: UserWarning: The figure layout has changed to tight
  plt.tight_layout()



=== Subject: Foreperiod_Sub2 ===
Trial counts: {'short': 324, 'long': 310, 'surprise': 74}
Sequence counts: {'long_short': 150, 'short_short': 148, 'short_long': 138, 'long_long': 131, 'surprise_long': 41, 'short_surprise': 39, 'long_surprise': 28, 'surprise_short': 26, 'surprise_surprise': 7}


...


=== Subject: Sub_06 ===
Trial counts: {'short': 310, 'long': 311, 'surprise': 69}
Sequence counts: {'short_short': 151, 'long_long': 144, 'long_short': 132, 'short_long': 131, 'surprise_long': 36, 'long_surprise': 35, 'short_surprise': 28, 'surprise_short': 27, 'surprise_surprise': 6}


C:\Users\bhdib\AppData\Local\Temp\ipykernel_25556\2751598477.py:69: UserWarning: The figure layout has changed to tight
  plt.tight_layout()



=== Subject: sub-01-foreperiod-12-3 ===
Trial counts: {'short': 334, 'long': 312, 'surprise': 64}
Sequence counts: {'short_short': 158, 'long_short': 147, 'short_long': 144, 'long_long': 135, 'surprise_long': 33, 'short_surprise': 32, 'long_surprise': 30, 'surprise_short': 29, 'surprise_surprise': 2}


C:\Users\bhdib\AppData\Local\Temp\ipykernel_25556\2751598477.py:69: UserWarning: The figure layout has changed to tight
  plt.tight_layout()



Saved: subjects_summary.csv → C:\Users\bhdib\OneDrive\Desktop\Foreperiod-Consolidated\Foreperiod-Consolidated\FIF\ANALYSIS
Saved: erp_metrics_per_subject.csv → C:\Users\bhdib\OneDrive\Desktop\Foreperiod-Consolidated\Foreperiod-Consolidated\FIF\ANALYSIS
Saved: erp_metrics_group.csv → C:\Users\bhdib\OneDrive\Desktop\Foreperiod-Consolidated\Foreperiod-Consolidated\FIF\ANALYSIS

All done. Check the ANALYSIS folder for CSVs and plots.
